In [39]:
import csv
import pickle
from datetime import datetime
from sklearn import preprocessing
import numpy as np
import random
from sklearn.preprocessing import OneHotEncoder
import sys

def csv2dicts(csvfile):
    data = []
    keys = []
    for row_index, row in enumerate(csvfile):
        if row_index == 0:
            keys = row
            print(row)
            continue
        # if row_index % 10000 == 0:
        #     print(row_index)
        data.append({key: value for key, value in zip(keys, row)})
    return data


def set_nan_as_string(data, replace_str='0'):
    for i, x in enumerate(data):
        for key, value in x.items():
            if value == '':
                x[key] = replace_str
        data[i] = x

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
PATH = '/content/drive/My Drive/Colab Notebooks/kaggle-rossmann-master/rossmann/rossmann/'
test_data_filename = PATH+"test.csv"
train_data_filename = PATH+"train.csv"
store_data_filename = PATH+"store.csv"
store_states_filename = PATH+'store_states.csv'

In [42]:
csvfile = open(train_data_filename)
train_data = csv.reader(csvfile, delimiter=',')
train_data = csv2dicts(train_data)
train_data = train_data[::-1]

print(train_data[:3])

['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']
[{'Store': '1115', 'DayOfWeek': '2', 'Date': '2013-01-01', 'Sales': '0', 'Customers': '0', 'Open': '0', 'Promo': '0', 'StateHoliday': 'a', 'SchoolHoliday': '1'}, {'Store': '1114', 'DayOfWeek': '2', 'Date': '2013-01-01', 'Sales': '0', 'Customers': '0', 'Open': '0', 'Promo': '0', 'StateHoliday': 'a', 'SchoolHoliday': '1'}, {'Store': '1113', 'DayOfWeek': '2', 'Date': '2013-01-01', 'Sales': '0', 'Customers': '0', 'Open': '0', 'Promo': '0', 'StateHoliday': 'a', 'SchoolHoliday': '1'}]


In [43]:
csvfile_test = open(test_data_filename)
test_data = csv.reader(csvfile_test, delimiter=',')
test_data = csv2dicts(test_data)
test_data = test_data[::-1]

print(test_data[:3])

['Id', 'Store', 'DayOfWeek', 'Date', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']
[{'Id': '41088', 'Store': '1115', 'DayOfWeek': '6', 'Date': '2015-08-01', 'Open': '1', 'Promo': '0', 'StateHoliday': '0', 'SchoolHoliday': '1'}, {'Id': '41087', 'Store': '1114', 'DayOfWeek': '6', 'Date': '2015-08-01', 'Open': '1', 'Promo': '0', 'StateHoliday': '0', 'SchoolHoliday': '0'}, {'Id': '41086', 'Store': '1113', 'DayOfWeek': '6', 'Date': '2015-08-01', 'Open': '1', 'Promo': '0', 'StateHoliday': '0', 'SchoolHoliday': '0'}]


In [44]:
csvfile = open(store_data_filename)
csvfile2 = open(store_states_filename)
store_data = csv.reader(csvfile, delimiter=',')
store_states_data = csv.reader(csvfile2, delimiter=',')

store_data = csv2dicts(store_data)
store_states_data = csv2dicts(store_states_data)
set_nan_as_string(store_data)
for index, val in enumerate(store_data):
    state = store_states_data[index]
    val['State'] = state['State']
    store_data[index] = val

print(store_data[:2])

['Store', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']
['Store', 'State']
[{'Store': '1', 'StoreType': 'c', 'Assortment': 'a', 'CompetitionDistance': '1270', 'CompetitionOpenSinceMonth': '9', 'CompetitionOpenSinceYear': '2008', 'Promo2': '0', 'Promo2SinceWeek': '0', 'Promo2SinceYear': '0', 'PromoInterval': '0', 'State': 'HE'}, {'Store': '2', 'StoreType': 'a', 'Assortment': 'a', 'CompetitionDistance': '570', 'CompetitionOpenSinceMonth': '11', 'CompetitionOpenSinceYear': '2007', 'Promo2': '1', 'Promo2SinceWeek': '13', 'Promo2SinceYear': '2010', 'PromoInterval': 'Jan,Apr,Jul,Oct', 'State': 'TH'}]


# Prepare features

In [45]:
random.seed(42)

def feature_list(record):
    dt = datetime.strptime(record['Date'], '%Y-%m-%d')
    store_index = int(record['Store'])
    year = dt.year
    month = dt.month
    day = dt.day
    day_of_week = int(record['DayOfWeek'])
    try:
        store_open = int(record['Open'])
    except:
        store_open = 1

    promo = int(record['Promo'])

    return [store_open,
            store_index,
            day_of_week,
            promo,
            year,
            month,
            day,
            store_data[store_index - 1]['State']
            ]

In [46]:
train_data_X = []
train_data_y = []

for record in train_data:
    if record['Sales'] != '0' and record['Open'] != '':
    #if record['Open'] != '':
        fl = feature_list(record)
        train_data_X.append(fl)
        train_data_y.append(int(record['Sales']))
print("Number of train datapoints: ", len(train_data_y))
print(min(train_data_y), max(train_data_y))

Number of train datapoints:  844338
46 41551


In [47]:
train_data_X[:10]

[[1, 1097, 2, 0, 2013, 1, 1, 'RP'],
 [1, 948, 2, 0, 2013, 1, 1, 'BW'],
 [1, 769, 2, 0, 2013, 1, 1, 'NW'],
 [1, 733, 2, 0, 2013, 1, 1, 'NW'],
 [1, 682, 2, 0, 2013, 1, 1, 'BE'],
 [1, 676, 2, 0, 2013, 1, 1, 'HE'],
 [1, 562, 2, 0, 2013, 1, 1, 'HB,NI'],
 [1, 530, 2, 0, 2013, 1, 1, 'SH'],
 [1, 512, 2, 0, 2013, 1, 1, 'BY'],
 [1, 494, 2, 0, 2013, 1, 1, 'BE']]

In [48]:
test_data_X = []
for record in test_data:
    fl = feature_list(record)
    test_data_X.append(fl)

In [49]:
test_data_X[:10]

[[1, 1115, 6, 0, 2015, 8, 1, 'HE'],
 [1, 1114, 6, 0, 2015, 8, 1, 'HH'],
 [1, 1113, 6, 0, 2015, 8, 1, 'SH'],
 [1, 1112, 6, 0, 2015, 8, 1, 'NW'],
 [1, 1111, 6, 0, 2015, 8, 1, 'NW'],
 [1, 1109, 6, 0, 2015, 8, 1, 'BY'],
 [1, 1107, 6, 0, 2015, 8, 1, 'BY'],
 [1, 1106, 6, 0, 2015, 8, 1, 'SH'],
 [1, 1105, 6, 0, 2015, 8, 1, 'NW'],
 [1, 1104, 6, 0, 2015, 8, 1, 'BY']]

In [50]:
full_X = np.array(train_data_X)
train_data_X = np.array(train_data_X)
full_X.shape

(844338, 8)

In [51]:
test_data_X = np.array(test_data_X)
test_data_X.shape

(41088, 8)

In [52]:
train_data_X[:, 0]

array(['1', '1', '1', ..., '1', '1', '1'], dtype='<U21')

In [53]:
les = []
for i in range(train_data_X.shape[1]):
    le = preprocessing.LabelEncoder()
    le.fit(full_X[:, i])
    les.append(le)
    train_data_X[:, i] = le.transform(train_data_X[:, i])
    if i > 0:
        test_data_X[:, i] = le.transform(test_data_X[:, i])

In [54]:
train_data_X.shape

(844338, 8)

In [55]:
train_data_X = train_data_X.astype(int)
train_data_y = np.array(train_data_y)
# les y  (train_data_X, train_data_y)

In [56]:
print(train_data_X[0], train_data_y[0])

[  0 109   1   0   0   0   0   7] 5961


In [57]:
np.random.seed(123)

sys.setrecursionlimit(10000)

train_ratio = 0.9
shuffle_data = False
one_hot_as_input = False
embeddings_as_input = False
save_embeddings = True
saved_embeddings_fname = "embeddings.pickle"  # set save_embeddings to True to create this file

(X, y) = train_data_X, train_data_y

num_records = len(X)
train_size = int(train_ratio * num_records)

In [58]:
if shuffle_data:
    print("Using shuffled data")
    sh = numpy.arange(X.shape[0])
    np.random.shuffle(sh)
    X = X[sh]
    y = y[sh]

if embeddings_as_input:
    print("Using learned embeddings as input")
    X = embed_features(X, saved_embeddings_fname)

if one_hot_as_input:
    print("Using one-hot encoding as input")
    enc = OneHotEncoder(sparse=False)
    enc.fit(X)
    X = enc.transform(X)

X_train = X[:train_size]
X_val = X[train_size:]
y_train = y[:train_size]
y_val = y[train_size:]

print(X_train.shape, X_val.shape)

(759904, 8) (84434, 8)


In [59]:
def sample(X, y, n):
    '''random samples'''
    num_row = X.shape[0]
    indices = np.random.randint(num_row, size=n)
    return X[indices, :], y[indices]


X_train, y_train = sample(X_train, y_train, 200000)  # Simulate data sparsity
print("Number of samples used for training: " + str(y_train.shape[0]))

Number of samples used for training: 200000


In [60]:
X_train.shape

(200000, 8)

In [61]:
import numpy
numpy.random.seed(123)
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn import neighbors
from sklearn.preprocessing import Normalizer

from keras.models import Sequential
from keras.models import Model as KerasModel
from keras.layers import Input, Dense, Activation, Reshape
from keras.layers import Concatenate
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint

import pickle


def embed_features(X, saved_embeddings_fname):
    # f_embeddings = open("embeddings_shuffled.pickle", "rb")
    f_embeddings = open(saved_embeddings_fname, "rb")
    embeddings = pickle.load(f_embeddings)

    index_embedding_mapping = {1: 0, 2: 1, 4: 2, 5: 3, 6: 4, 7: 5}
    X_embedded = []

    (num_records, num_features) = X.shape
    for record in X:
        embedded_features = []
        for i, feat in enumerate(record):
            feat = int(feat)
            if i not in index_embedding_mapping.keys():
                embedded_features += [feat]
            else:
                embedding_index = index_embedding_mapping[i]
                embedded_features += embeddings[embedding_index][feat].tolist()

        X_embedded.append(embedded_features)

    return numpy.array(X_embedded)


def split_features(X):
    X_list = []

    store_index = X[..., [1]]
    X_list.append(store_index)

    day_of_week = X[..., [2]]
    X_list.append(day_of_week)

    promo = X[..., [3]]
    X_list.append(promo)

    year = X[..., [4]]
    X_list.append(year)

    month = X[..., [5]]
    X_list.append(month)

    day = X[..., [6]]
    X_list.append(day)

    State = X[..., [7]]
    X_list.append(State)

    return X_list


class Model(object):

    def evaluate(self, X_val, y_val):
        assert(min(y_val) > 0)
        guessed_sales = self.guess(X_val)
        relative_err = numpy.absolute((y_val - guessed_sales) / y_val)
        result = numpy.sum(relative_err) / len(y_val)
        return result


class LinearModel(Model):

    def __init__(self, X_train, y_train, X_val, y_val):
        super().__init__()
        self.clf = linear_model.LinearRegression()
        self.clf.fit(X_train, numpy.log(y_train))
        print("Result on validation data: ", self.evaluate(X_val, y_val))

    def guess(self, feature):
        return numpy.exp(self.clf.predict(feature))


class RF(Model):

    def __init__(self, X_train, y_train, X_val, y_val):
        super().__init__()
        self.clf = RandomForestRegressor(n_estimators=200, verbose=True, max_depth=35, min_samples_split=2,
                                         min_samples_leaf=1)
        self.clf.fit(X_train, numpy.log(y_train))
        print("Result on validation data: ", self.evaluate(X_val, y_val))

    def guess(self, feature):
        return numpy.exp(self.clf.predict(feature))


class SVM(Model):

    def __init__(self, X_train, y_train, X_val, y_val):
        super().__init__()
        self.X_train = X_train
        self.y_train = y_train
        self.__normalize_data()
        self.clf = SVR(kernel='linear', degree=3, gamma='auto', coef0=0.0, tol=0.001,
                       C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)

        self.clf.fit(self.X_train, numpy.log(self.y_train))
        print("Result on validation data: ", self.evaluate(X_val, y_val))

    def __normalize_data(self):
        self.scaler = StandardScaler()
        self.X_train = self.scaler.fit_transform(self.X_train)

    def guess(self, feature):
        return numpy.exp(self.clf.predict(feature))


class XGBoost(Model):

    def __init__(self, X_train, y_train, X_val, y_val):
        super().__init__()
        dtrain = xgb.DMatrix(X_train, label=numpy.log(y_train))
        evallist = [(dtrain, 'train')]
        param = {'nthread': -1,
                 'max_depth': 7,
                 'eta': 0.02,
                 'silent': 1,
                 'objective': 'reg:linear',
                 'colsample_bytree': 0.7,
                 'subsample': 0.7}
        num_round = 3000
        self.bst = xgb.train(param, dtrain, num_round, evallist)
        print("Result on validation data: ", self.evaluate(X_val, y_val))

    def guess(self, feature):
        dtest = xgb.DMatrix(feature)
        return numpy.exp(self.bst.predict(dtest))


class HistricalMedian(Model):

    def __init__(self, X_train, y_train, X_val, y_val):
        super().__init__()
        self.history = {}
        self.feature_index = [1, 2, 3, 4]
        for x, y in zip(X_train, y_train):
            key = tuple(x[self.feature_index])
            self.history.setdefault(key, []).append(y)
        print("Result on validation data: ", self.evaluate(X_val, y_val))

    def guess(self, features):
        features = numpy.array(features)
        features = features[:, self.feature_index]
        guessed_sales = [numpy.median(self.history[tuple(feature)]) for feature in features]
        return numpy.array(guessed_sales)


class KNN(Model):

    def __init__(self, X_train, y_train, X_val, y_val):
        super().__init__()
        self.normalizer = Normalizer()
        self.normalizer.fit(X_train)
        self.clf = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance', p=1)
        self.clf.fit(self.normalizer.transform(X_train), numpy.log(y_train))
        print("Result on validation data: ", self.evaluate(self.normalizer.transform(X_val), y_val))

    def guess(self, feature):
        return numpy.exp(self.clf.predict(self.normalizer.transform(feature)))


class NN_with_EntityEmbedding(Model):

    def __init__(self, X_train, y_train, X_val, y_val):
        super().__init__()
        self.epochs = 10
        self.checkpointer = ModelCheckpoint(filepath="best_model_weights.hdf5", verbose=1, save_best_only=True)
        self.max_log_y = max(numpy.max(numpy.log(y_train)), numpy.max(numpy.log(y_val)))
        self.__build_keras_model()
        self.fit(X_train, y_train, X_val, y_val)

    def preprocessing(self, X):
        X_list = split_features(X)
        return X_list

    def __build_keras_model(self):
        input_store = Input(shape=(1,))
        output_store = Embedding(1115, 10, name='store_embedding')(input_store)
        output_store = Reshape(target_shape=(10,))(output_store)

        input_dow = Input(shape=(1,))
        output_dow = Embedding(7, 6, name='dow_embedding')(input_dow)
        output_dow = Reshape(target_shape=(6,))(output_dow)

        input_promo = Input(shape=(1,))
        output_promo = Dense(1)(input_promo)

        input_year = Input(shape=(1,))
        output_year = Embedding(3, 2, name='year_embedding')(input_year)
        output_year = Reshape(target_shape=(2,))(output_year)

        input_month = Input(shape=(1,))
        output_month = Embedding(12, 6, name='month_embedding')(input_month)
        output_month = Reshape(target_shape=(6,))(output_month)

        input_day = Input(shape=(1,))
        output_day = Embedding(31, 10, name='day_embedding')(input_day)
        output_day = Reshape(target_shape=(10,))(output_day)

        input_germanstate = Input(shape=(1,))
        output_germanstate = Embedding(12, 6, name='state_embedding')(input_germanstate)
        output_germanstate = Reshape(target_shape=(6,))(output_germanstate)

        input_model = [input_store, input_dow, input_promo,
                       input_year, input_month, input_day, input_germanstate]

        output_embeddings = [output_store, output_dow, output_promo,
                             output_year, output_month, output_day, output_germanstate]

        output_model = Concatenate()(output_embeddings)
        output_model = Dense(1000, kernel_initializer="uniform")(output_model)
        output_model = Activation('relu')(output_model)
        output_model = Dense(500, kernel_initializer="uniform")(output_model)
        output_model = Activation('relu')(output_model)
        output_model = Dense(1)(output_model)
        output_model = Activation('sigmoid')(output_model)

        self.model = KerasModel(inputs=input_model, outputs=output_model)

        self.model.compile(loss='mean_absolute_error', optimizer='adam')

    def _val_for_fit(self, val):
        val = numpy.log(val) / self.max_log_y
        return val

    def _val_for_pred(self, val):
        return numpy.exp(val * self.max_log_y)

    def fit(self, X_train, y_train, X_val, y_val):
        self.model.fit(self.preprocessing(X_train), self._val_for_fit(y_train),
                       validation_data=(self.preprocessing(X_val), self._val_for_fit(y_val)),
                       epochs=self.epochs, batch_size=128,
                       # callbacks=[self.checkpointer],
                       )
        # self.model.load_weights('best_model_weights.hdf5')
        print("Result on validation data: ", self.evaluate(X_val, y_val))

    def guess(self, features):
        features = self.preprocessing(features)
        result = self.model.predict(features).flatten()
        return self._val_for_pred(result)


class NN(Model):

    def __init__(self, X_train, y_train, X_val, y_val):
        super().__init__()
        self.epochs = 10
        self.checkpointer = ModelCheckpoint(filepath="best_model_weights.hdf5", verbose=1, save_best_only=True)
        self.max_log_y = max(numpy.max(numpy.log(y_train)), numpy.max(numpy.log(y_val)))
        self.__build_keras_model()
        self.fit(X_train, y_train, X_val, y_val)

    def __build_keras_model(self):
        self.model = Sequential()
        self.model.add(Dense(1000, kernel_initializer="uniform", input_dim=1183))
        self.model.add(Activation('relu'))
        self.model.add(Dense(500, kernel_initializer="uniform"))
        self.model.add(Activation('relu'))
        self.model.add(Dense(1))
        self.model.add(Activation('sigmoid'))

        self.model.compile(loss='mean_absolute_error', optimizer='adam')

    def _val_for_fit(self, val):
        val = numpy.log(val) / self.max_log_y
        return val

    def _val_for_pred(self, val):
        return numpy.exp(val * self.max_log_y)

    def fit(self, X_train, y_train, X_val, y_val):
        self.model.fit(X_train, self._val_for_fit(y_train),
                       validation_data=(X_val, self._val_for_fit(y_val)),
                       epochs=self.epochs, batch_size=128,
                       # callbacks=[self.checkpointer],
                       )
        # self.model.load_weights('best_model_weights.hdf5')
        print("Result on validation data: ", self.evaluate(X_val, y_val))

    def guess(self, features):
        result = self.model.predict(features).flatten()
        return self._val_for_pred(result)

In [62]:
models = []

print("Fitting NN_with_EntityEmbedding...")
for i in range(5):
    models.append(NN_with_EntityEmbedding(X_train, y_train, X_val, y_val))

Fitting NN_with_EntityEmbedding...
Epoch 1/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.0144 - val_loss: 0.0120
Epoch 2/10
1563/1563 [==============================] - 7s 5ms/step - loss: 0.0094 - val_loss: 0.0114
Epoch 3/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.0086 - val_loss: 0.0113
Epoch 4/10
1563/1563 [==============================] - 7s 5ms/step - loss: 0.0080 - val_loss: 0.0102
Epoch 5/10
1563/1563 [==============================] - 7s 5ms/step - loss: 0.0077 - val_loss: 0.0101
Epoch 6/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.0074 - val_loss: 0.0102
Epoch 7/10
1563/1563 [==============================] - 7s 5ms/step - loss: 0.0073 - val_loss: 0.0102
Epoch 8/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.0071 - val_loss: 0.0099
Epoch 9/10
1563/1563 [==============================] - 7s 5ms/step - loss: 0.0069 - val_loss: 0.0104
Epoch 10/10
1563/1563 [========================

In [75]:
test_data_X[:, 1:].shape

(41088, 7)

In [77]:
test_data_j=test_data_X.astype(np.float)

In [78]:
test_data_j[:, 1:]

array([[130.,   5.,   0., ...,  10.,   0.,   4.],
       [129.,   5.,   0., ...,  10.,   0.,   5.],
       [128.,   5.,   0., ...,  10.,   0.,   8.],
       ...,
       [782.,   3.,   1., ...,  11.,   8.,   8.],
       [338.,   3.,   1., ...,  11.,   8.,   6.],
       [  0.,   3.,   1., ...,  11.,   8.,   4.]])

In [79]:
predictions_norm = models[0].model.predict(np.hsplit(test_data_j[:, 1:], 7))

In [83]:
y_pred_test = np.exp(predictions_norm * models[0].max_log_y)

In [84]:
y_pred_test[(test_data_j[:, 0] == 0)] = 0

In [85]:
import pandas as pd
sample_csv = pd.read_csv(PATH+'sample_submission.csv')

In [86]:
sample_csv['Sales'] = y_pred_test

In [87]:
sample_csv.to_csv(f'submision_original.csv', index=False)

In [88]:
sample_csv.head()

,Id,Sales
0,1,8338.788086
1,2,25477.736328
2,3,6862.466797
3,4,8058.010254
4,5,3168.840332


In [ ]:
# print("Fitting NN...")
# for i in range(5):
#     models.append(NN(X_train, y_train, X_val, y_val))

# print("Fitting RF...")
# models.append(RF(X_train, y_train, X_val, y_val))

# print("Fitting KNN...")
# models.append(KNN(X_train, y_train, X_val, y_val))

# print("Fitting XGBoost...")
# models.append(XGBoost(X_train, y_train, X_val, y_val))


if save_embeddings:
    model = models[0].model
    store_embedding = model.get_layer('store_embedding').get_weights()[0]
    dow_embedding = model.get_layer('dow_embedding').get_weights()[0]
    year_embedding = model.get_layer('year_embedding').get_weights()[0]
    month_embedding = model.get_layer('month_embedding').get_weights()[0]
    day_embedding = model.get_layer('day_embedding').get_weights()[0]
    german_states_embedding = model.get_layer('state_embedding').get_weights()[0]
    with open(saved_embeddings_fname, 'wb') as f:
        pickle.dump([store_embedding, dow_embedding, year_embedding,
                     month_embedding, day_embedding, german_states_embedding], f, -1)


def evaluate_models(models, X, y):
    assert(min(y) > 0)
    guessed_sales = numpy.array([model.guess(X) for model in models])
    mean_sales = guessed_sales.mean(axis=0)
    relative_err = numpy.absolute((y - mean_sales) / y)
    result = numpy.sum(relative_err) / len(y)
    return result


print("Evaluate combined models...")
print("Training error...")
r_train = evaluate_models(models, X_train, y_train)
print(r_train)

print("Validation error...")
r_val = evaluate_models(models, X_val, y_val)
print(r_val)

Evaluate combined models...
Training error...


In [74]:
X_val

array([[  0,  89,   5, ...,   7,  11,   4],
       [  0,  88,   5, ...,   7,  11,   1],
       [  0,  87,   5, ...,   7,  11,   6],
       ...,
       [  0, 338,   4, ...,   9,  24,   6],
       [  0, 227,   4, ...,   9,  24,  11],
       [  0,   0,   4, ...,   9,  24,   4]])